# Langchain Agents 
This notebook demonstrates how to create and use agents in Langchain, including the use of tools like DuckDuckGoSearch for real-time information retrieval.

At the end of this notebook, we can see how to create a ReAct agent from a prompt template and use it with a memory system to maintain chat history.



In [ ]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq


In [ ]:
# load environment variables from .env file
load_dotenv()

In [ ]:
# Get llms models via the `langchain_groq` package
llm_model = ChatGroq(
    model="llama-3.3-70b-versatile", #llama-3.3-70b-versatile",
    temperature=.5,
    max_tokens=512,
)

## Langchain Tools

The [third-party tools](https://python.langchain.com/docs/integrations/tools/) from the langchain-community ecosytem allow us to extent the capabilities of our agents

In [ ]:
# from langchain_community.tools import DuckDuckGoSearchRun
from ddgs import DDGS
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

`DuckDuckGoSearchRun` allows us to perform web searches and retrieve information in real-time, which can be very useful for agents that need to answer questions based on current data.


In [ ]:
#api_wrapper = DuckDuckGoSearchAPIWrapper(region="en-en",max_results=10)
#search = DuckDuckGoSearchRun()#,api_wrapper=api_wrapper)

def ddg_search(query: str):
    with DDGS() as ddgs:
        return ddgs.text(query, max_results=5)
search = ddg_search

In [ ]:
results = search("Prime Italian Minister")  

In [ ]:
results

In [ ]:
# Define a tool for the DuckDuckGo search
from langchain.tools import Tool
search_tool = Tool(
    name="DuckDuckGoSearch",
    func=ddg_search,
    description="Use this tool when you need to search for real-time information from DuckDuckGo."
)

In [ ]:
from langchain.agents import initialize_agent, AgentType


In [ ]:
# Initialize the agent with the DuckDuckGo search tool
agent = initialize_agent(
    tools=[search_tool],
    llm=llm_model,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=7,
    agent_name="DuckDuckGoSearchAgent",
    agent_description="An agent that can search the web using DuckDuckGoSearch and summarize findings clearly."
)

# Now we can use the agent to answer questions
results = agent.invoke("Who is the Italian Prime Minister?")  # Example query to the agent

### Create a an Agent with DuckDuckGoSearch with Memory
In this section, we will create an agent that can search the web using DuckDuckGoSearch and maintain a memory of previous interactions. 

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


In [ ]:

# Build a chat prompt that includes conversation history so the agent can resolve references.
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("assistant", "{agent_scratchpad}"),
])


The react_template instructs the laguage model to:
1. __Think__ about how to solve a user's question
2. __Use tools__ (actions) to gather mor information or perform actions
3. __Observe results__ and update the its thoughts
4. Eventually, reach a __Final Answer__

The following is the explanation of the `react_template` used to create the agent:

```python
react_template = """
'Answer the following questions as best you can. You have access to the following tools:

{tools}
```
+ Introduce the agent and tells it what tools it can use
+ {tools} is a placeholder for the tools that will be used by the agent

```text
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
```

This is the strict __format__ the LLM must follow to:

* Show its reasoning (Thought)

* Choose and execute a tool (Action)

* Feed it input (Action Input)

* Observe results (Observation)

* Loop until it gets to the final answer

```text
Begin!
Question: {input}
Thought:{agent_scratchpad}'
"""
```
+ `{input}` is the actual user question.

+ `{agent_scratchpad}` holds any intermediate steps already completed (e.g., previous Thoughts, Actions, Observations).

+ The agent continues from this point.

In [ ]:
# prompt_template defined above with chat history support


In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
# Simple session store to keep track of chat history per conversation
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [ ]:
# Create the agent 
agent = create_react_agent(llm_model, [search_tool], prompt_template)
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    handle_parsing_errors=True,
)


In [ ]:
# Wrap the agent executor with message history
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [ ]:
# Now we can use the agent with chat history to answer questions
agent_with_chat_history.invoke(
    {"input": "How many people live in italy?"},
    config={"configurable": {"session_id": "1"}},
)

In [ ]:
# Let's ask another question to see how the agent uses the chat history
agent_with_chat_history.invoke(
    {"input": "what is their national anthem called?"},
    config={"configurable": {"session_id": "1"}},
)

### Exercise
Create or Select some tools from the [langchain-community tools](https://docs.langchain.com/oss/python/integrations/tools) and create a ReAct agent that can use them to solve certain tasks.